First of all, make sure your environment has installed the latest version of [🤗 Optimum Graphcore](https://github.com/huggingface/optimum-graphcore).

In [1]:
#! pip install optimum[graphcore]

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /home/jincheng/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


Then you need to install Git-LFS. Uncomment the following instructions:

In [3]:
# !apt install git-lfs

Let's print out the versions of Transformers and Optimum Graphcore:

In [4]:
import transformers
import optimum.graphcore

print(transformers.__version__)
print(optimum.graphcore.__version__)

4.20.0
0.3.1


# Fine-tuning a language model

In this notebook, we'll see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modeling tasks. We will cover two types of language modeling tasks which are:

- Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence.

![Widget inference representing the causal language modeling task](images/causal_language_modeling.png)

- Masked language modeling: the model has to predict some tokens that are masked in the input. It still has access to the whole sentence, so it can use the tokens before and after the tokens masked to predict their value.

![Widget inference representing the masked language modeling task](images/masked_language_modeling.png)

We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `IPUTrainer` API to fine-tune a model on it.

## Preparing the dataset

For each of those tasks, we will use the [Wikitext 2]() dataset as an example. You can load it very easily with the 🤗 Datasets library.

In [5]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

Reusing dataset wikitext (/localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

In [6]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split first, then give an index:

In [7]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [8]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(datasets["train"])

,text
0,"On the eastern part of the island , on the Stupa Hill , there is a small group of caves that house Buddhist monuments . This hill is named after the religious Stupa monument that they display . One of the two caves is incomplete , while the other contains a Stupa made in brick . \n"
1,"Many reviews also noted the game 's high difficulty . Both Giancarlo Varanini of GameSpot and Craig Harris of IGN found that the game 's bosses have very unpredictable attack patterns , thus making the battles extremely challenging . Harris additionally observed a heavy amount of trial @-@ and @-@ error for the levels themselves where the player must die several times before completing each one . He concluded , "" [ ... ] It 's really the way Mega Man games have always been ... and to be honest , with all of the annoying little deaths in the game , there 's always that sensation after every failure that you 've learned the challenge , and perseverance definitely prevails in this game "" . \n"
2,
3,
4,= = Pitching style = = \n
5,"In cool , moist areas ranging from 4 @,@ 800 feet ( 1 @,@ 500 m ) to 6 @,@ 800 feet ( 2 @,@ 100 m ) , Douglas fir is predominant . Lodgepole pine and quaking aspen may be found alongside Douglas fir in cooler areas , both moist and dry , but particularly where frost pockets form . Understories in this forest type are dominated by mountain maple , mountain ash , and blue huckleberry in moister areas and white spirea , common snowberry , elk sedge , and pinegrass in drier areas . Between 3 @,@ 400 feet ( 1 @,@ 000 m ) and 6 @,@ 500 feet ( 2 @,@ 000 m ) in the moist northern parts of the forest , grand fir is predominant and western larch is one of the first trees to become established during ecological succession following disturbances , whereas understories consist of mountain maple , mountain ash , blue huckleberry , and mallow ninebark . Subalpine fir dominates from 4 @,@ 800 feet ( 1 @,@ 500 m ) to 7 @,@ 500 feet ( 2 @,@ 300 m ) along with mountain maple , serviceberry , Scouler 's willow , Sitka alder , menziesia , Utah honeysuckle , and mountain ash . \n"
6,
7,
8,
9,"Hurricane Brenda originated from a tropical wave that moved off the western coast of Africa on August 9 ; however , the initial wave quickly weakened upon entering the Atlantic Ocean . By August 13 , the wave began to regenerate as it passed through the Lesser Antilles . Several days later , convection associated with the system consolidated into a central , organized mass and on August 18 , the system had become sufficiently organized to be declared a tropical depression while situated near the Yucatán Channel . Early the next day , the depression strengthened into Tropical Storm Brenda as it made landfall in the northern portion of the Yucatán Peninsula . After moving inland , a strong ridge of high pressure over Texas forced the storm to take an unusual track , eventually leading it to enter the Bay of Campeche on August 20 . \n"


As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

## Causal Language modeling

For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:
```
part of text 1
```
or 
```
end of text 1 [BOS_TOKEN] beginning of text 2
```
depending on whether they span over several of the original texts in the dataset or not. The labels will be the same as the inputs, shifted to the left.

We will use the [`gpt2`](https://huggingface.co/gpt2) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=causal-lm) instead as long as that model is supported by Optimum Graphcore:

In [10]:
model_checkpoint = "gpt2"

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [11]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [12]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [13]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-a905f8f4f8e1ff8c.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-c85b036588100830.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-076b841cba8ec22e.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-a9c0084ce6652ff2.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-b5e44e42be02c6d5.arrow


...

Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-44dafc1874256e00.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-7118e9551cd8a39e.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-055c05c16303427e.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-fdcb7c92c76c63cc.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-bdbb965f4f4270b7.arrow


If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [14]:
tokenized_datasets["train"][1]

{'input_ids': [796, 569, 18354, 7496, 17740, 6711, 796, 220, 198],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. Here we take a bit less at just 128.

In [15]:
# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:

In [16]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

Also note that by default, the `map` method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of `block_size` every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

In [17]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-4def8e4cc5a47525.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-e8545d43a25bf09e.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-b3aa11eb0e3a7f1b.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-8d950158b8aeb5a1.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-e61b83f298c9e682.arrow


...

Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-f39626b72910330c.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-2c31b2a75ff60bab.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-0e93e92292b9a4e7.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-49ca3731827c0534.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-abbef1ff4dfc06bd.arrow


And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

In [18]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' game and follows the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ". \n The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries, along with Valkyria Chronicles II director Takeshi Oz'

Now that the data has been cleaned, we're ready to instantiate our `IPUTrainer`. Firstly we will create a model:

In [19]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

Then we need to define the `IPUConfig`, which is a class that specifies attributes and configuration parameters to compile and put the model on the device. We initialize it with one config name or path, which we set earlier. We also get the model configuration from the model name set earlier and initialize our model using that config:

In [20]:
from optimum.graphcore import IPUConfig
ipu_config_name = "Graphcore/gpt2-small-ipu"
ipu_config = IPUConfig.from_pretrained(
    ipu_config_name,
)

And some `IPUTrainingArguments`:

In [21]:
from optimum.graphcore import IPUConfig, IPUTrainer, IPUTrainingArguments

In [22]:
micro_batch_size = 1
gradient_accumulation_steps = 16
pod_type = "pod16"

model_name = model_checkpoint.split("/")[-1]
training_args = IPUTrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=micro_batch_size,
    per_device_eval_batch_size=micro_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    pod_type=pod_type,
    dataloader_drop_last=True,
    logging_steps=10,
    push_to_hub=True,
)

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/gpt-finetuned-wikitext2"` or `"huggingface/gpt-finetuned-wikitext2"`).

We pass along all of those to the `IPUTrainer` class:

In [23]:
trainer = IPUTrainer(
    model=model,
    ipu_config=ipu_config,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

Overriding IPU config: gradient_accumulation_steps=16
-------------------- Device Allocation --------------------
Token Embedding     --> IPU 0
Position Embedding  --> IPU 1
Layer 0             --> IPU 1
Layer 1             --> IPU 1
Layer 2             --> IPU 1
Layer 3             --> IPU 1
Layer 4             --> IPU 2
Layer 5             --> IPU 2
Layer 6             --> IPU 2
Layer 7             --> IPU 2
Layer 8             --> IPU 3
Layer 9             --> IPU 3
Layer 10            --> IPU 3
Layer 11            --> IPU 3
Head                --> IPU 0
-----------------------------------------------------------
/localdata/jincheng/optimum-graphcore/notebooks/gpt2-finetuned-wikitext2 is already a clone of https://huggingface.co/Jinchen/gpt2-finetuned-wikitext2. Make sure you pull the latest changes with `repo.git_pull()`.


And we can train our model:

In [24]:
trainer.train()

Compiling Model...
/localdata/jincheng/sdks/poplar_sdk-ubuntu_20_04-2.6.0+1066-459b49240b/2.6.0+1066_poptorch/lib/python3.8/site-packages/transformers/models/gpt2/modeling_gpt2.py:798: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if batch_size <= 0:
[16:31:36.989] [poptorch:cpp] [warning] %3112 : Long(1, strides=[128], requires_grad=0, device=cpu) = aten::fill_(%3111, %988) # /localdata/jincheng/sdks/poplar_sdk-ubuntu_20_04-2.6.0+1066-459b49240b/2.6.0+1066_poptorch/lib/python3.8/site-packages/optimum/graphcore/models/gpt2/modeling_gpt2.py:219:0: torch.int64 is not supported natively on IPU, loss of range/precision may occur. We will only warn on the first instance.
Graph compilation: 100%|████████████████████████████████████████████████████████████████████████████████

  0%|          | 0/435 [00:00<?, ?it/s]

{'loss': 4.4582, 'learning_rate': 1.9540229885057475e-05, 'epoch': 0.07}
{'loss': 4.9344, 'learning_rate': 1.908045977011494e-05, 'epoch': 0.14}
{'loss': 4.5391, 'learning_rate': 1.8620689655172415e-05, 'epoch': 0.21}
{'loss': 4.3457, 'learning_rate': 1.8160919540229885e-05, 'epoch': 0.28}
{'loss': 4.409, 'learning_rate': 1.770114942528736e-05, 'epoch': 0.34}
{'loss': 4.2258, 'learning_rate': 1.7241379310344828e-05, 'epoch': 0.41}
{'loss': 4.3043, 'learning_rate': 1.6781609195402298e-05, 'epoch': 0.48}
{'loss': 4.0494, 'learning_rate': 1.632183908045977e-05, 'epoch': 0.55}
{'loss': 4.123, 'learning_rate': 1.586206896551724e-05, 'epoch': 0.62}
{'loss': 4.1041, 'learning_rate': 1.540229885057471e-05, 'epoch': 0.69}
{'loss': 4.1533, 'learning_rate': 1.4942528735632185e-05, 'epoch': 0.76}
{'loss': 4.1063, 'learning_rate': 1.4482758620689657e-05, 'epoch': 0.83}
{'loss': 4.0793, 'learning_rate': 1.4022988505747128e-05, 'epoch': 0.9}
{'loss': 4.1203, 'learning_rate': 1.3563218390804598e-05, '

Compiling Model...

Graph compilation:   0%|                                                                                                                                                                                                                                                                                     | 0/100 [00:00<?]2022-08-10T15:33:31.393783Z popart:devicex 3944840.3944840 W: The `debug.retainDebugInformation` engine option was implicitly set to `true`. The default will change to `false` in a future release. Set it to `true` explicitly if you want to query debug information (for example, by calling `Session::getReport`).

2022-08-10T15:33:41.791462Z popart:devicex 3944840.3944840 W: The `debug.retainDebugInformation` engine option was implicitly set to `true`. The default will change to `false` in a future release. Set it to `true` explicitly if you want to query debug information (for example, by calling `Session::getReport`).
Graph compilation: 100%|█████████████

  0%|          | 0/96 [00:00<?, ?it/s]

...

  0%|          | 0/96 [00:00<?, ?it/s]

{'eval_loss': 3.677734375, 'eval_runtime': 4.1919, 'eval_samples_per_second': 458.021, 'eval_steps_per_second': 22.901, 'epoch': 3.0}




Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 381.2406, 'train_samples_per_second': 146.049, 'train_steps_per_second': 1.141, 'train_loss': 4.042941810344828, 'epoch': 3.0}


TrainOutput(global_step=435, training_loss=4.042941810344828, metrics={'train_runtime': 381.2406, 'train_samples_per_second': 146.049, 'train_steps_per_second': 1.141, 'train_loss': 4.042941810344828, 'epoch': 3.0})

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [25]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1931
  Batch size = 20


  0%|          | 0/96 [00:00<?, ?it/s]

Perplexity: 39.56


You can now upload the result of the training to the Hub, just execute this instruction:

In [26]:
trainer.push_to_hub()

Saving model checkpoint to gpt2-finetuned-wikitext2
-------------------- Device Allocation --------------------
Token Embedding     --> IPU 0
Position Embedding  --> IPU 1
Layer 0             --> IPU 1
Layer 1             --> IPU 1
Layer 2             --> IPU 1
Layer 3             --> IPU 1
Layer 4             --> IPU 2
Layer 5             --> IPU 2
Layer 6             --> IPU 2
Layer 7             --> IPU 2
Layer 8             --> IPU 3
Layer 9             --> IPU 3
Layer 10            --> IPU 3
Layer 11            --> IPU 3
Head                --> IPU 0
-----------------------------------------------------------
Configuration saved in gpt2-finetuned-wikitext2/ipu_config.json


Upload file pytorch_model.bin:   0%|          | 32.0k/249M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 2.61k/2.61k [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/Jinchen/gpt2-finetuned-wikitext2
   e98c9fd..388d006  main -> main

To https://huggingface.co/Jinchen/gpt2-finetuned-wikitext2
   388d006..29e1ff7  main -> main



'https://huggingface.co/Jinchen/gpt2-finetuned-wikitext2/commit/388d0064671d8ad670f579ebf369744414946b2f'

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sgugger/my-awesome-model")
```

## Masked language modeling

For masked language modeling (MLM) we are going to use the same preprocessing as before for our dataset with one additional step: we will randomly mask some tokens (by replacing them by `[MASK]`) and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens).

We will use the [`roberta-base`](https://huggingface.co/roberta-base) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=masked-lm) instead as long as that model is supported by Optimum Graphcore:

In [27]:
model_checkpoint = "roberta-base"

We can apply the same tokenization function as before, we just need to update our tokenizer to use the checkpoint we just picked:

In [28]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-06decd28049867ac.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-e62be675d4904a02.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-bd23ae6b57286f49.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-2a610943f9150202.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-de3557adcfe704d2.arrow


...

Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-a7a01ae198bfb1b0.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-2704ab1d2b50dcf7.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-271ab04cbda3da27.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-9e0ebcc7c9c49fba.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-7b3bde059a0260e8.arrow


And like before, we group texts together and chunk them in samples of length `block_size`. You can skip that step if your dataset is composed of individual sentences.

In [29]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-34ea058173b68686.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-18b261bbaf922c63.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-d138ade87cd64ee3.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-4d7c53ff86dc4d71.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-54fa8991b59954fc.arrow


...

Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-122dc890c7262fda.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-bcc962a25e01ceb3.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-d617a4e4dad2fba4.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-56be1a8e5a66cfc7.arrow


Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-2356e831aca79392.arrow


The rest is very similar to what we had, with two exceptions. First we use a model suitable for masked LM:

In [30]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

We redefine our `IPUConfig`:

In [31]:
ipu_config_name = "Graphcore/roberta-base-ipu"
ipu_config = IPUConfig.from_pretrained(
    ipu_config_name,
)

loading configuration file https://huggingface.co/Graphcore/roberta-base-ipu/resolve/main/ipu_config.json from cache at /localdata/jincheng/huggingface/transformers/9c7d358d1e5475a12e6c8bd64a6da4a3f9675d59ef23a01a87b32c74eb3f1519.973f2c7663c486b6c9ea637f58eddfec86b600c10d70d04893f39214419000ab
IPUConfig {
  "decompose_grad_sum": false,
  "device_iterations": 1,
  "embedding_serialization_factor": 0,
  "enable_half_first_order_momentum": true,
  "enable_half_partials": true,
  "executable_cache_dir": "./exe_cache",
  "execute_encoder_on_cpu_for_generation": false,
  "gradient_accumulation_steps": 16,
  "inference_device_iterations": 5,
  "inference_replication_factor": {
    "default": 1,
    "pod16": 4,
    "pod32": 8,
    "pod4": 1,
    "pod64": 16,
    "pod8": 2
  },
  "ipus_per_replica": 4,
  "layers_per_ipu": [
    0,
    4,
    4,
    4
  ],
  "matmul_proportion": [
    0.25
  ],
  "optimizer_state_offchip": true,
  "optimum_version": "1.3.0",
  "output_mode": "final",
  "profile_

We redefine our `IPUTrainingArguments`:

In [32]:
micro_batch_size = 1
gradient_accumulation_steps = 16
pod_type = "pod16"

model_name = model_checkpoint.split("/")[-1]
training_args = IPUTrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=micro_batch_size,
    per_device_eval_batch_size=micro_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    pod_type=pod_type,
    dataloader_drop_last=True,
    logging_steps=10,
    push_to_hub=True,
)

The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Like before, the last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-wikitext2"` or `"huggingface/bert-finetuned-wikitext2"`).

Finally, we use a special `data_collator`. The `data_collator` is a function that is responsible of taking the samples and batching them in tensors. In the previous example, we had nothing special to do, so we just used the default for this argument. Here we want to do the random-masking. We could do it as a pre-processing step (like the tokenization) but then the tokens would always be masked the same way at each epoch. By doing this step inside the `data_collator`, we ensure this random masking is done in a new way each time we go over the data.

To do this masking for us, the library provides a `DataCollatorForLanguageModeling`. We can adjust the probability of the masking:

In [33]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Then we just have to pass everything to `IPUTrainer` and begin training:

In [34]:
trainer = IPUTrainer(
    model=model,
    ipu_config=ipu_config,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

Overriding IPU config: gradient_accumulation_steps=16
-------------------- Device Allocation --------------------
Embedding  --> IPU 0
Encoder 0  --> IPU 1
Encoder 1  --> IPU 1
Encoder 2  --> IPU 1
Encoder 3  --> IPU 1
Encoder 4  --> IPU 2
Encoder 5  --> IPU 2
Encoder 6  --> IPU 2
Encoder 7  --> IPU 2
Encoder 8  --> IPU 3
Encoder 9  --> IPU 3
Encoder 10 --> IPU 3
Encoder 11 --> IPU 3
LM Head    --> IPU 0
-----------------------------------------------------------
Cloning https://huggingface.co/Jinchen/roberta-base-finetuned-wikitext2 into local empty directory.


Download file pytorch_model.bin:   0%|          | 16.6k/238M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 2.61k/2.61k [00:00<?, ?B/s]

Clean file training_args.bin:  38%|###8      | 1.00k/2.61k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/238M [00:00<?, ?B/s]

In [35]:
trainer.train()

Compiling Model...
/localdata/jincheng/sdks/poplar_sdk-ubuntu_20_04-2.6.0+1066-459b49240b/2.6.0+1066_poptorch/lib/python3.8/site-packages/optimum/graphcore/models/roberta/modeling_roberta.py:169: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  max_number_of_masked_tokens = int(labels.size(1) * 0.25)
/localdata/jincheng/sdks/poplar_sdk-ubuntu_20_04-2.6.0+1066-459b49240b/2.6.0+1066_poptorch/lib/python3.8/site-packages/optimum/graphcore/modeling_utils.py:409: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  num_classes = int(sequence.shape[1])
Graph compilatio

  0%|          | 0/900 [00:00<?, ?it/s]

{'loss': 2.2265, 'learning_rate': 1.977777777777778e-05, 'epoch': 0.03}
{'loss': 2.0082, 'learning_rate': 1.9555555555555557e-05, 'epoch': 0.07}
{'loss': 1.959, 'learning_rate': 1.9333333333333333e-05, 'epoch': 0.1}
{'loss': 1.5749, 'learning_rate': 1.9111111111111113e-05, 'epoch': 0.13}
{'loss': 1.8838, 'learning_rate': 1.888888888888889e-05, 'epoch': 0.17}
{'loss': 1.8602, 'learning_rate': 1.866666666666667e-05, 'epoch': 0.2}
{'loss': 1.8956, 'learning_rate': 1.8444444444444448e-05, 'epoch': 0.23}
{'loss': 1.7629, 'learning_rate': 1.8222222222222224e-05, 'epoch': 0.27}
{'loss': 1.6989, 'learning_rate': 1.8e-05, 'epoch': 0.3}
{'loss': 1.6899, 'learning_rate': 1.7777777777777777e-05, 'epoch': 0.33}
{'loss': 1.7745, 'learning_rate': 1.7555555555555556e-05, 'epoch': 0.37}
{'loss': 1.4696, 'learning_rate': 1.7333333333333336e-05, 'epoch': 0.4}
{'loss': 1.6169, 'learning_rate': 1.7111111111111112e-05, 'epoch': 0.43}
{'loss': 1.6538, 'learning_rate': 1.688888888888889e-05, 'epoch': 0.47}
{'

Compiling Model...

Graph compilation:   0%|                                                                                                                                                                                                                                                                                     | 0/100 [00:00<?]2022-08-10T15:42:27.066500Z popart:devicex 3944840.3944840 W: The `debug.retainDebugInformation` engine option was implicitly set to `true`. The default will change to `false` in a future release. Set it to `true` explicitly if you want to query debug information (for example, by calling `Session::getReport`).

2022-08-10T15:42:29.796408Z popart:devicex 3944840.3944840 W: The `debug.retainDebugInformation` engine option was implicitly set to `true`. The default will change to `false` in a future release. Set it to `true` explicitly if you want to query debug information (for example, by calling `Session::getReport`).
Graph compilation: 100%|█████████████

  0%|          | 0/99 [00:00<?, ?it/s]

...

  0%|          | 0/99 [00:00<?, ?it/s]

{'eval_loss': 1.4970703125, 'eval_runtime': 8.8026, 'eval_samples_per_second': 224.934, 'eval_steps_per_second': 11.247, 'epoch': 3.0}




Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 409.5129, 'train_samples_per_second': 140.655, 'train_steps_per_second': 2.198, 'train_loss': 1.6625336244371203, 'epoch': 3.0}


TrainOutput(global_step=900, training_loss=1.6625336244371203, metrics={'train_runtime': 409.5129, 'train_samples_per_second': 140.655, 'train_steps_per_second': 2.198, 'train_loss': 1.6625336244371203, 'epoch': 3.0})

Like before, we can evaluate our model on the validation set. The perplexity is much lower than for the CLM objective because for the MLM objective, we only have to make predictions for the masked tokens (which represent 15% of the total here) while having access to the rest of the tokens. It's thus an easier task for the model.

In [36]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1988
  Batch size = 20


  0%|          | 0/99 [00:00<?, ?it/s]

Perplexity: 4.49


You can now upload the result of the training to the Hub, just execute this instruction:

In [37]:
trainer.push_to_hub()

Saving model checkpoint to roberta-base-finetuned-wikitext2
-------------------- Device Allocation --------------------
Embedding  --> IPU 0
Encoder 0  --> IPU 1
Encoder 1  --> IPU 1
Encoder 2  --> IPU 1
Encoder 3  --> IPU 1
Encoder 4  --> IPU 2
Encoder 5  --> IPU 2
Encoder 6  --> IPU 2
Encoder 7  --> IPU 2
Encoder 8  --> IPU 3
Encoder 9  --> IPU 3
Encoder 10 --> IPU 3
Encoder 11 --> IPU 3
LM Head    --> IPU 0
-----------------------------------------------------------
Configuration saved in roberta-base-finetuned-wikitext2/ipu_config.json


Upload file pytorch_model.bin:   0%|          | 32.0k/238M [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/Jinchen/roberta-base-finetuned-wikitext2
   abc7832..7d5d830  main -> main

To https://huggingface.co/Jinchen/roberta-base-finetuned-wikitext2
   7d5d830..2ff266f  main -> main



'https://huggingface.co/Jinchen/roberta-base-finetuned-wikitext2/commit/7d5d83069b47e1d7fe20e73cffef36ba0818941e'

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("sgugger/my-awesome-model")
```